In [1]:
#copied from https://github.com/haydarozler/bike-sharing-prediction-with-RNN/blob/master/BikeSharing-V3.ipynb
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import pandas as pd;
import matplotlib.pyplot as plt
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense, LSTM, RNN, SimpleRNN, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.optimizers import Adam
from keras.layers.core import Activation
from keras.callbacks import LambdaCallback
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [8]:
def get_3d_data(data):
    in_seq1 = array(datatrain['holiday'])
    in_seq2 = array(datatrain['workingday'])
    in_seq3 = array(datatrain['temp'])
    in_seq4 = array(datatrain['atemp'])
    in_seq5 = array(datatrain['hum'])
    in_seq6 = array(datatrain['windspeed'])
    in_seq7 = array(datatrain['weekday_0'])
    in_seq8 = array(datatrain['weekday_1'])
    in_seq9 = array(datatrain['weekday_2'])
    in_seq10 = array(datatrain['weekday_3'])
    in_seq11 = array(datatrain['weekday_4'])
    in_seq12 = array(datatrain['weekday_5'])
    in_seq13 = array(datatrain['weekday_6'])
    in_seq14 = array(datatrain['weathersit_1'])
    in_seq15 = array(datatrain['weathersit_2'])
    in_seq16 = array(datatrain['weathersit_3'])
    out_seq = array(datatrain['cntscl'])
    
    in_seq1 = in_seq1.reshape((len(in_seq1), 1))
    in_seq2 = in_seq2.reshape((len(in_seq2), 1))
    in_seq3 = in_seq3.reshape((len(in_seq3), 1))
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    in_seq5 = in_seq5.reshape((len(in_seq5), 1))
    in_seq6 = in_seq6.reshape((len(in_seq6), 1))
    in_seq7 = in_seq7.reshape((len(in_seq7), 1))
    in_seq8 = in_seq8.reshape((len(in_seq8), 1))
    in_seq9 = in_seq9.reshape((len(in_seq9), 1))
    in_seq10 = in_seq10.reshape((len(in_seq10), 1))
    in_seq11 = in_seq11.reshape((len(in_seq11), 1))
    in_seq12 = in_seq12.reshape((len(in_seq12), 1))
    in_seq13 = in_seq13.reshape((len(in_seq13), 1))
    in_seq14 = in_seq14.reshape((len(in_seq14), 1))
    in_seq15 = in_seq15.reshape((len(in_seq15), 1))
    in_seq16 = in_seq16.reshape((len(in_seq16), 1))
    out_seq = out_seq.reshape((len(out_seq), 1))
    
    data_feed = hstack((in_seq1, in_seq2, in_seq3, in_seq4, in_seq5, in_seq6, in_seq7, in_seq8, in_seq9, in_seq10, in_seq11, in_seq12, in_seq13, in_seq14, in_seq15, in_seq16, out_seq))
    return data_feed, out_seq

In [3]:
dataset = pd.read_csv('data/uci_bike_sharing/day.csv')
dataset.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [4]:
one_hot = pd.get_dummies(dataset['weekday'], prefix='weekday')
dataset = dataset.join(one_hot)
one_hot = pd.get_dummies(dataset['weathersit'], prefix='weathersit')
dataset = dataset.join(one_hot)
one_hot = pd.get_dummies(dataset['mnth'], prefix='mnth')
dataset = dataset.join(one_hot)

#scale the counts 
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(array(dataset['cnt']).reshape(len(dataset['cnt']), 1))
series = pd.DataFrame(scaled)
series.columns = ['cntscl']
dataset = pd.merge(dataset, series, left_index=True, right_index=True)

In [5]:
number_of_test_data = 50
number_of_holdout_data = 50
number_of_training_data = len(dataset) - number_of_holdout_data - number_of_test_data
print ("Total, Train, Val, Test:", len(dataset), number_of_training_data, number_of_test_data, number_of_holdout_data)

Total, Train, Val, Test: 731 631 50 50


In [6]:
datatrain = dataset[:number_of_training_data]
datatest = dataset[-(number_of_test_data+number_of_holdout_data):-number_of_holdout_data]
datahold = dataset[-number_of_holdout_data:]

In [9]:
datatrain_feed, out_seq_train = get_3d_data(datatrain)
datatest_feed, out_seq_test = get_3d_data(datatest)
datahold_feed, out_seq_hold = get_3d_data(datahold)

In [10]:
n_features = datatrain_feed.shape[1]
n_input = 10
generator_train = TimeseriesGenerator(datatrain_feed, out_seq_train, length=n_input, batch_size=len(datatrain_feed))
generator_test = TimeseriesGenerator(datatest_feed, out_seq_test, length=n_input, batch_size=1)
generator_hold = TimeseriesGenerator(datahold_feed, out_seq_hold, length=n_input, batch_size=1)

In [11]:
datatest.head()
print("Timesteps, Features:", n_input, n_features)

Timesteps, Features: 10 17


In [12]:
adam = Adam(lr=0.0001)
model = Sequential()
model.add(SimpleRNN(4, activation='relu', input_shape=(n_input, n_features), return_sequences = False))
model.add(Dense(1, activation='softplus'))
model.compile(optimizer=adam, loss='poisson')

In [13]:
score = model.fit_generator(generator_train, epochs=20, verbose=2, validation_data=generator_test)


Epoch 1/20
 - 26s - loss: 0.9358 - val_loss: 1.0542
Epoch 2/20
 - 22s - loss: 0.9355 - val_loss: 1.0543
Epoch 3/20
 - 15s - loss: 0.9353 - val_loss: 1.0544
Epoch 4/20
 - 44s - loss: 0.9351 - val_loss: 1.0544
Epoch 5/20
 - 50s - loss: 0.9349 - val_loss: 1.0545
Epoch 6/20
 - 15s - loss: 0.9347 - val_loss: 1.0546
Epoch 7/20
 - 23s - loss: 0.9345 - val_loss: 1.0547
Epoch 8/20
 - 11s - loss: 0.9343 - val_loss: 1.0547
Epoch 9/20
 - 24s - loss: 0.9341 - val_loss: 1.0548
Epoch 10/20
 - 26s - loss: 0.9339 - val_loss: 1.0549
Epoch 11/20
 - 14s - loss: 0.9337 - val_loss: 1.0550
Epoch 12/20
 - 16s - loss: 0.9334 - val_loss: 1.0551
Epoch 13/20
 - 7s - loss: 0.9332 - val_loss: 1.0551
Epoch 14/20
 - 4s - loss: 0.9330 - val_loss: 1.0552
Epoch 15/20
 - 18s - loss: 0.9328 - val_loss: 1.0553
Epoch 16/20
 - 17s - loss: 0.9326 - val_loss: 1.0553
Epoch 17/20
 - 14s - loss: 0.9324 - val_loss: 1.0554
Epoch 18/20
 - 12s - loss: 0.9322 - val_loss: 1.0555
Epoch 19/20
 - 18s - loss: 0.9320 - val_loss: 1.0556
Epo

In [29]:
test_score = model.predict_generator(generator_hold)

In [30]:
test_score_scl = scaler.inverse_transform(test_score)
df_result = pd.DataFrame({'Actual' : [], 'Prediction' : []})
for i in range(len(datahold)):
    df_result = df_result.append({'Actual': datahold.iloc[i]['cnt'], 'Prediction': test_score_scl[i][0]}, ignore_index=True)
df_result

,Actual,Prediction
0,6269.0,6305.144531
1,4094.0,10117.052734
2,5495.0,6246.955078
3,5445.0,7712.425293
4,5698.0,6245.139160
5,5629.0,5248.716797
6,4669.0,5256.649902
7,5499.0,7406.697754
8,5634.0,9783.560547
9,5146.0,9401.915039
